In [5]:
%pip install h2o sklearn

Defaulting to user installation because normal site-packages is not writeable
Processing ./.cache/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897/sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-1.0.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import sklearn

In [7]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_312-debug"; OpenJDK Runtime Environment (build 1.8.0_312-debug-b07); OpenJDK 64-Bit Server VM (build 25.312-b07-debug, mixed mode)
  Starting server from /home/mk7516/.local/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /state/partition1/job-19046308/tmpr3jczzsu
  JVM stdout: /state/partition1/job-19046308/tmpr3jczzsu/h2o_mk7516_started_from_python.out
  JVM stderr: /state/partition1/job-19046308/tmpr3jczzsu/h2o_mk7516_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,20 days
H2O_cluster_name:,H2O_from_python_mk7516_v36emp
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.541 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [8]:
import pandas as pd
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(data_id=name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  y_train = y_train.to_frame(name="class")
  y_test = y_test.to_frame(name="class")
  test = pd.concat([X_test, y_test], axis=1, join='inner')
  train = pd.concat([X_train, y_train], axis=1, join='inner')
  test = h2o.H2OFrame(test)
  train = h2o.H2OFrame(train)
  return (test, train)

In [9]:
test, train = dataSetup(41163)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [10]:
train.shape

(7500, 2001)

In [11]:
x = train.columns
y = "class"
x.remove(y)

In [12]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [13]:
aml = H2OAutoML(max_runtime_secs = 3600)
aml.train(x=x, y=y, training_frame=train)
# train = h2o.H2OFrame(train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_1_20220504_131316


Model Summary: 


,,number_of_trees
0,,30.0




ModelMetricsMultinomial: xgboost
** Reported on train data. **

MSE: 0.4731870783543977
RMSE: 0.6878859486531163
LogLoss: 1.1993214787109494
Mean Per-Class Error: 0.3737777892623183
AUC: NaN
AUCPR: NaN
Multinomial auc values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).
Multinomial auc_pr values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,0,1,2,3,4,Error,Rate
0,900.0,122.0,156.0,148.0,158.0,0.393531,"584 / 1,484"
1,119.0,837.0,160.0,151.0,169.0,0.417131,"599 / 1,436"
2,124.0,95.0,1023.0,129.0,169.0,0.335714,"517 / 1,540"
3,131.0,98.0,154.0,906.0,174.0,0.380725,"557 / 1,463"
4,133.0,104.0,176.0,126.0,1038.0,0.341788,"539 / 1,577"
5,1407.0,1256.0,1669.0,1460.0,1708.0,0.372800,"2,796 / 7,500"



Top-5 Hit Ratios: 


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,training_auc,training_pr_auc
0,,2022-05-04 13:14:16,59.294 sec,0.0,0.800000,1.609438,0.804933,NaN,NaN
1,,2022-05-04 13:14:18,1 min 1.584 sec,5.0,0.772131,1.493495,0.587733,NaN,NaN
2,,2022-05-04 13:14:20,1 min 3.064 sec,10.0,0.753090,1.421331,0.529467,NaN,NaN
3,,2022-05-04 13:14:21,1 min 4.636 sec,15.0,0.732598,1.349279,0.484800,NaN,NaN
4,,2022-05-04 13:14:23,1 min 6.080 sec,20.0,0.715603,1.290837,0.434400,NaN,NaN
5,,2022-05-04 13:14:24,1 min 7.477 sec,25.0,0.701208,1.243193,0.407733,NaN,NaN
6,,2022-05-04 13:14:26,1 min 8.828 sec,30.0,0.687886,1.199321,0.372800,NaN,NaN



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,V1620,57.334614,1.000000,0.004188
1,V1674,49.830849,0.869123,0.003640
2,V869,49.098217,0.856345,0.003586
3,V847,48.002899,0.837241,0.003506
4,V1875,46.875565,0.817579,0.003424
5,V1666,46.472942,0.810556,0.003394
6,V1561,45.706623,0.797191,0.003338
7,V1074,43.714771,0.762450,0.003193
8,V1596,43.710049,0.762368,0.003192
9,V1422,41.471508,0.723324,0.003029



See the whole table with table.as_data_frame()


In [15]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_per_class_error,logloss,rmse,mse
XGBoost_1_AutoML_1_20220504_131316,0.785981,1.7086,0.802295,0.643677
StackedEnsemble_AllModels_3_AutoML_1_20220504_131316,0.788652,1.63778,0.799209,0.638735
XGBoost_3_AutoML_1_20220504_131316,0.78883,1.63774,0.800506,0.64081
StackedEnsemble_AllModels_2_AutoML_1_20220504_131316,0.78902,1.62475,0.798894,0.638231
StackedEnsemble_BestOfFamily_3_AutoML_1_20220504_131316,0.790046,1.61748,0.799409,0.639054
StackedEnsemble_AllModels_1_AutoML_1_20220504_131316,0.790056,1.61806,0.79926,0.638816
XGBoost_grid_1_AutoML_1_20220504_131316_model_4,0.790647,1.63071,0.79996,0.639935
GBM_grid_1_AutoML_1_20220504_131316_model_1,0.791395,1.62594,0.800432,0.640691
XGBoost_grid_1_AutoML_1_20220504_131316_model_1,0.791816,1.67391,0.801619,0.642593
GBM_grid_1_AutoML_1_20220504_131316_model_2,0.792031,1.6092,0.799816,0.639706
